In [1]:
%load_ext cython

In [2]:
import numpy as np
import pandas as pd
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import RBF
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error

### PLS1 original

In [3]:
def PLS1(X, Y):
    r = np.linalg.matrix_rank(X)
    E_h = X
    F_h = Y.reshape(-1,1)
    W = {}
    B = {}
    P = {}
    u = np.zeros(Y.shape[0],dtype=np.float32)
    w = np.zeros([X.shape[1],1],dtype=np.float32)
    t = np.zeros([X.shape[0], 1],dtype=np.float32)
#     t_ = np.zeros([X.shape[0], 1],dtype=np.float32)
    p = np.zeros([X.shape[1], 1],dtype=np.float32)
#     q = np.zeros([Y.shape[1], 1],dtype=np.float32)
#     q_ = np.zeros([Y.shape[1], 1],dtype=np.float32)
    b = 0.0
    for i in range(r):
#         print(X.shape)
        #step 1
        u = Y
        #step 2
        w = np.dot(X.T, u) / np.dot(u.T, u)
        #step 3
        w = w/np.linalg.norm(w)
        #step 4
        t = np.dot(X, w)/np.dot(w.T, w)
        #step5-8 omitted
        #step 9
        p = np.dot(X.T, t) / np.dot(t.T, t)
        #step 10
        p = p/np.linalg.norm(p)
        #step 11
        t = t* np.linalg.norm(p)
        #step 12
        w = w * np.linalg.norm(p)
        #step 13
        b = np.dot(u.T, t)/np.dot(t.T, t)
#         print(b.shape)
        # Calculation of the residuals
        t = t.reshape((-1,1))
        p = p.reshape((-1,1))
        E_h = E_h - np.dot(t,p.T)
        F_h = F_h - b*t
#         print(F_h.shape)
        #Replace X and Y
        X = E_h
        Y = F_h
        #update W and B
        W[i] = w
        B[i] = b
        P[i] = p
    return W,B,P
    

In [4]:
def predict_original(X,Y,X_test):
    W,B,P = PLS1(X,Y)
    r = np.linalg.matrix_rank(X)
    Q = np.ones((1,r))
    E_h = X_test
    y_pred = np.zeros((X_test.shape[0],1))
    for i in range(r):
        t_hat = E_h @ W[i]
        E_h = E_h - t_hat @ P[i].T
        y_pred = y_pred + B[i] * t_hat
    return y_pred
        

### optimized PLS1 using Cython

In [5]:
%%cython -a

import numpy as np
cimport numpy as np
from libc.stdio cimport printf
from libc.math cimport sqrt
from cython.parallel import prange, parallel
import cython
cimport cython

ctypedef np.double_t DTYPE_t
ctypedef np.int64_t TTYPE_t

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)



cpdef DTYPE_t dot_1d(np.ndarray[DTYPE_t,ndim = 2] v1, np.ndarray[DTYPE_t,ndim = 2] v2):
    cdef DTYPE_t result = 0
    cdef int i = 0
    cdef int length = v1.shape[0]
    cdef double el1 = 0
    cdef double el2 = 0
    for i in range(length):
        el1 = v1[i,0]
        el2 = v2[0,i]
        result += el1*el2
    return result

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cpdef double norm_1d(double[:,:] v1):
    cdef double result = 0
    cdef int i = 0
    cdef int length = v1.shape[0]
    for i in range(length):
        result += v1[i,0]*v1[i,0]
    result = sqrt(result)
    return result

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cpdef np.ndarray[DTYPE_t, ndim=2] scalar_multiply(DTYPE_t a, np.ndarray[DTYPE_t, ndim=2] b):
    cdef np.ndarray[DTYPE_t, ndim=2] mat = b.copy()
    cdef TTYPE_t blen = b.shape[0]
    cdef TTYPE_t bwid = b.shape[1]
    for i in range(blen):
        for j in range(bwid):
            mat[i, j] = a*b[i, j]
    return mat

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cpdef np.ndarray[DTYPE_t, ndim=1] scalar_division(np.ndarray[DTYPE_t, ndim=1] vec, DTYPE_t sca):
    cdef np.ndarray[DTYPE_t, ndim=1] mat = vec.copy()
    cdef TTYPE_t blen = vec.shape[0]
    cdef int i
    with cython.nogil, parallel():
        for i in prange(blen):
            mat[i] = vec[i]/sca
    return mat


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cpdef np.ndarray[DTYPE_t, ndim=2] scalar_division_1d2d(np.ndarray[DTYPE_t, ndim=2] vec, DTYPE_t sca):
    cdef np.ndarray[DTYPE_t, ndim=2] mat = vec.copy()
    cdef TTYPE_t blen = vec.shape[0]
    cdef TTYPE_t i
    with cython.nogil, parallel():
        for i in prange(blen):
            mat[i,0] = vec[i, 0]/sca
    return mat

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cpdef np.ndarray[DTYPE_t, ndim=2] minus_2d(np.ndarray[DTYPE_t, ndim=2] A, np.ndarray[DTYPE_t, ndim=2] B):
    cdef np.ndarray[DTYPE_t, ndim=2] mat = A.copy()
    cdef int i, j
    with cython.nogil, parallel():
        for i in prange(A.shape[0]):
            for j in prange(A.shape[1]):
                mat[i,j] = A[i,j] - B[i,j]
    return mat

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cpdef PLS_cython(np.ndarray[DTYPE_t, ndim=2] X, np.ndarray[DTYPE_t, ndim=2] Y, int r):
    cdef np.ndarray[DTYPE_t, ndim=2] E_h = X.copy()
    cdef np.ndarray[DTYPE_t, ndim=2] F_h = Y.copy()
    W = {}
    B = {}
    P = {}
    cdef np.ndarray[DTYPE_t, ndim=2] u = Y.copy()
    cdef np.ndarray[DTYPE_t, ndim=2] w = np.zeros([X.shape[1],1])
    cdef np.ndarray[DTYPE_t, ndim=2] t = np.zeros([X.shape[0], 1])
    cdef np.ndarray[DTYPE_t, ndim=2] p = np.zeros([X.shape[1], 1])
    cdef DTYPE_t b = 0.0
    cdef int i
    
    
    for i in range(r):
        u = Y
        w = scalar_division_1d2d(np.dot(X.T, u), dot_1d(u,u.T))

        #step 3
        w = w/norm_1d(w)

        #step 4

        t = scalar_division_1d2d(np.dot(X, w),dot_1d(w, w.T))
        #step5-8 omitted
        #step 9
        p = scalar_division_1d2d(np.dot(X.T, t),dot_1d(t,t.T))
        p_norm = norm_1d(p)
        #step 10
        p = p/p_norm       
        #step 11
        t = t* p_norm
        #step 12
        w = w * p_norm
        #step 13
        b = np.dot(u.T, t)/dot_1d(t,t.T)
#         print(b.shape)
        # Calculation of the residuals

        E_h = minus_2d(E_h,np.dot(t,p.T))
        F_h = minus_2d(F_h,scalar_multiply(b,t))
#         print(F_h.shape)
        #Replace X and Y
        X = E_h
        Y = F_h
        #update W and B
        W[i] = w
        B[i] = b
        P[i] = p
    return W,B,P

cpdef predict_cython(np.ndarray[DTYPE_t, ndim=2] X, np.ndarray[DTYPE_t, ndim=2] Y,np.ndarray[DTYPE_t, ndim=2] X_test,int r):
    W,B,P = PLS_cython(X,Y,r)
    cdef np.ndarray[DTYPE_t, ndim=2] E_h = X_test.copy()
    cdef np.ndarray[DTYPE_t, ndim=2] y_pred = np.zeros((X_test.shape[0],1))
    cdef np.ndarray[DTYPE_t, ndim=2] t_hat = np.zeros((X_test.shape[0],1))
    cdef int i,j
    for i in range(r):
        t_hat = np.dot(E_h, W[i])
        E_h = E_h - np.dot(t_hat, P[i].T)
        for j in range(y_pred.shape[0]):
            y_pred[j,0] = y_pred[j,0] + B[i] * t_hat[j,0]
    return y_pred[:,0]
    
    

### GPR

In [6]:
def predict_GPR(X_train,y_train,X_test,y_test):
    kernel = DotProduct() + WhiteKernel()
    # kernel = RBF()
    gpr = GaussianProcessRegressor(kernel=kernel).fit(X_train, y_train)
    y_pred = gpr.predict(X_test)
    return y_pred

### OLS

In [7]:
def predict_OLS(X_train,y_train,X_test,y_test):
    model = sm.OLS(y_train,sm.add_constant(X_train))
    results = model.fit()
    y_pred = results.predict(sm.add_constant(X_test))
    return y_pred

### Simulated Data

In [8]:
import pandas as pd
'''
np.random.seed(9856)
x1 = np.random.normal(1, .2, 100)
x2 = np.random.normal(5, .4, 100)
x3 = np.random.normal(12, .8, 100)
'''

x1 = np.linspace(0, 10,100)
x2 = np.linspace(-5, 15,100)
x3 = np.linspace(-20, -15,100)


def generate_sim(x1, x2, x3):
    sim_data = {'x1': x1,
                'x2': x2,
                'x3': x3,
                'x4': 5 * x1,
                'x5': 2 * x2,
                'x6': 4 * x3, 
                'x7': 6 * x1,
                'x8': 5 * x2,
                'x9': 4 * x3,
                'x10': np.random.rand() * x1,
                'x11': np.random.rand() * x2,
                'x12': np.random.rand() * x3,
                'x13': np.random.rand() * x1,
                'x14': np.random.rand() * x2,
                'x15': np.random.rand() * x3,
                'x16': np.random.rand() * x1,
                'x17': np.random.rand() * x1,
                'x18': np.random.rand() * x2,
                'x19': np.random.rand() * x3,
                'y0': 3 * x2 + 3 * x3,
                'y1': 6 * x1 + 3 * x3,
                'y2': 7 * x2 + 2 * x1}

    # convert data to csv file
    data = pd.DataFrame(sim_data)

    sim_predictors = data.drop(['y0', 'y1', 'y2'], axis=1).columns.tolist()
    sim_values = ['y0']

    pred = data[sim_predictors].values
    val = data[sim_values].values

    return pred, val


X_test, y_test = generate_sim(x1, x2, x3)

# pred = pred.astype(np.float32)
# val = val.astype(np.float32)
# val_opt = val.reshape(-1,1)

# test_x1 = np.random.normal(1, .2, 3000).astype(np.float32)
# test_x2 = np.random.normal(5, .4, 3000).astype(np.float32)
# test_x3 = np.random.normal(12, .8, 3000).astype(np.float32)
test_x1 = np.random.normal(1, .2, 5000)
test_x2 = np.random.normal(5, .4, 5000)
test_x3 = np.random.normal(12, .8, 5000)

X_train, y_train = generate_sim(test_x1, test_x2, test_x3)


# pred_test = pred_test.astype(np.float32)
# pred_val = pred_val.astype(np.float32)


### Test Results

#### Original PLS1

In [9]:
%timeit y_pred_ori = predict_original(X_train, y_train,X_test)

3.59 ms ± 251 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
y_pred_ori  = predict_original(X_train, y_train,X_test)
y_pred_ori = y_pred_ori.reshape((y_pred_ori.shape[0]))

In [11]:
mean_squared_error(y_test, y_pred_ori)

1.1593387769561891

#### Optimized PLS1 using Cython

In [12]:
r = np.linalg.matrix_rank(X_train)

In [13]:
%timeit y_pred_cython = predict_cython(X_train,y_train,X_test,r)

1.58 ms ± 70.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [14]:
y_pred_cython = predict_cython(X_train,y_train,X_test,r)
y_pred_cython = y_pred_cython.reshape((y_pred_cython.shape[0]))

In [15]:
mean_squared_error(y_test, y_pred_cython)

1.1621948808943714

#### GPR

In [16]:
y_pred_GPR = predict_GPR(X_train,y_train,X_test,y_test)

In [17]:
mean_squared_error(y_test, y_pred_GPR)

1.16219487629266

In [18]:
%timeit predict_GPR(X_train,y_train,X_test,y_test)

13.1 s ± 142 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### OLS

In [19]:
y_pred_OLS= predict_OLS(X_train,y_train,X_test,y_test)

In [20]:
model = sm.OLS(y_train,sm.add_constant(X_train))
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 8.641e+30
Date:                Wed, 01 May 2019   Prob (F-statistic):               0.00
Time:                        00:10:34   Log-Likelihood:             1.4735e+05
No. Observations:                5000   AIC:                        -2.947e+05
Df Residuals:                    4996   BIC:                        -2.947e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -5.507e-14   1.07e-14     -5.153      0.000    -7.6e-14   -3.41e-14
x1          5.551e-17   4.26e-17      1.302      0.193   -2.81e-17    1.39e-16
x2             0.0966   4.36e-17   2.22e+15      0.000       0.097       0.097
x3             0.0877   1.93e-17   4.54e+15      0.000       0.088       0.088
x4          9.992e-16   2.13e-16      4.686      0.000    5.81e-16    1.42e-15
x5             0.1931   8.72e-17   2.22e+15      0.000       0.193       0.193
x6             0.3507   7.72e-17   4.54e+15      0.000       0.351       0.351
x7          1.027e-15   2.56e-16      4.014      0.000    5.25e-16    1.53e-15
x8             0.4828   2.18e-16   2.22e+15      0.000       0.483       0.483
x9             0.3507   7.72e-17   4.54e+15      0.000       0.351       0.351
x10         2.168e-17   5.03e-18      4.314      0.000    1.18e-17    3.15e-17
x11            0.0007   3.23e-19   2.22e+15      0.000       0.001       0.001
x12            0.0692   1.52e-17   4.54e+15      0.000       0.069       0.069
x13         1.214e-16   4.25e-17      2.859      0.004    3.82e-17    2.05e-16
x14            0.0419   1.89e-17   2.22e+15      0.000       0.042       0.042
x15            0.0540   1.19e-17   4.54e+15      0.000       0.054       0.054
x16         1.171e-17   2.88e-18      4.071      0.000    6.07e-18    1.73e-17
x17          2.22e-16   4.17e-17      5.329      0.000     1.4e-16    3.04e-16
x18            0.0905   4.09e-17   2.22e+15      0.000       0.091       0.091
x19            0.0408   8.98e-18   4.54e+15      0.000       0.041       0.041
==============================================================================
Omnibus:                        2.318   Durbin-Watson:                   0.073
Prob(Omnibus):                  0.314   Jarque-Bera (JB):                2.245
Skew:                           0.016   Prob(JB):                        0.325
Kurtosis:                       2.901   Cond. No.                     1.87e+19
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 8.26e-32. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [21]:
mean_squared_error(y_test, y_pred_OLS)

1.162194880894308

In [22]:
%timeit predict_OLS(X_train,y_train,X_test,y_test)

2.06 ms ± 10.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Real Data

In [23]:
wine_data = pd.read_csv("winequality-red.csv")
wine_data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [29]:
wine_d = wine_data.drop(["quality"], axis = 1)
wine_l = wine_data["quality"]

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
X_train, X_test, y_train, y_test = train_test_split(wine_d, wine_l, test_size=0.1, random_state=42)
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

#### Original PLS1

In [41]:
p_ori = predict_original(X_train, y_train,X_test)

In [42]:
mean_squared_error(y_test, p_ori)

0.4006952673584756

In [43]:
%timeit predict_original(X_train, y_train,X_test)

1.83 ms ± 70.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Cythonized PLS1

In [32]:
r = np.linalg.matrix_rank(X_train)
y_train = y_train.reshape([-1,1]).astype(np.double)
y_pred = predict_cython(X_train,y_train,X_test,r)

In [33]:
mean_squared_error(y_test, y_pred)

0.3841657740684029

In [37]:
%timeit predict_cython(X_train,y_train,X_test,r)

1.82 ms ± 17.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### OLS

In [34]:
ols_p = predict_OLS(X_train,y_train,X_test,y_test)

In [35]:
mean_squared_error(y_test, ols_p)

0.38368768129098707

In [36]:
%timeit predict_OLS(X_train,y_train,X_test,y_test)

710 µs ± 9.86 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### GPR

In [38]:
y_pred_GPR = predict_GPR(X_train,y_train,X_test,y_test)

In [39]:
mean_squared_error(y_test, y_pred_GPR)

0.3859786119630064

In [40]:
%timeit predict_GPR(X_train,y_train,X_test,y_test)

2.16 s ± 39.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
